In [1]:
%pip install datasets
%pip install nltk
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Part 0. Data Prepraration

In [2]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset ['train']
validation_dataset = dataset ['validation']
test_dataset = dataset ['test']

/Users/janellekoh/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Part 1. Preparing Word Embeddings

#### (a) What is the size of the vocabulary formed from your training data?

In [3]:
import nltk
import numpy as np
nltk.download('punkt')
vocab = set()
for text in train_dataset['text']:
    ls = nltk.word_tokenize(text)
    for word in ls:
        vocab.add(word)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/janellekoh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
print("Size of the vocabulary:", len(vocab))

Size of the vocabulary: 18030


#### (b) We use OOV (out-of-vocabulary) to refer to those words appeared in the training data but not in the Word2vec (or Glove) dictionary. How many OOV words exist in your training data?

In [5]:
import gensim.downloader as api
for key in api.info()['models'].keys():
    print(key)

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [6]:
model = api.load("glove-wiki-gigaword-100")

In [7]:
oov_words = set()
for word in vocab:
    if word not in model:
        oov_words.add(word)

print("Number of OOV words:", len(oov_words))

Number of OOV words: 1867


#### (c) The existence of the OOV words is one of the well-known limitations of Word2vec (or Glove). Without using any transformer-based language models (e.g., BERT, GPT, T5), what do you think is the best strategy to mitigate such limitation? Implement your solution in your source code. Show the corresponding code snippet.

In [8]:
# Group any words that are not in the model into a single token
def wordtovec(word):
    if word in model:
        return model[word]
    else:
        return np.zeros(model.vector_size)